In [1]:
# Standard library imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from datetime import timedelta

### I. Data Preparation
Our original dataset contains 150,346 entries of businesses recorded on Yelp, each record containing attributes such as ‘name’, ‘address’, ‘latitude’, ‘longitude’, ‘review_count’, with a labels column of ‘stars’.

Since we want our model to focus on predicting the success of restaurants, we must only keep the businesses that are labeled as ‘Restaurants’. Additionally, we must drop the columns that are irrelevant and those that will bias/skew our models, such as ‘name’ and ‘business_id’. We are choosing to deal with NaNs by dropping all records with NaN values and further downsizing our dataset by dropping all businesses that are no longer open.

In [2]:
# Read the data file

df = pd.read_json('data/yelp_academic_dataset_business.json', lines=True)
print("Shape of the data frame:", df.shape)

Shape of the data frame: (150346, 14)


In [3]:
# Drop all records with missing values and irrelevant columns
df = df.dropna()
df = df.drop(columns=['name', 'address', 'city'])

print("Shape of the modified data frame:", df.shape)

Shape of the modified data frame: (117618, 11)


In [4]:
# Keep only businesses that are restaurants
df = df[df['categories'].str.contains('Restaurants')]

# Keep only businesses that are still open (not permanently closed)
df = df[df['is_open']==1]

# Drop the is_open column (irrelevant)
df = df.drop(columns='is_open')

print("Shape of the modified data frame:", df.shape)

Shape of the modified data frame: (31357, 10)


In [5]:
# Parse JSON data in attributes and hours columns to individual feature columns

df = df.join(pd.json_normalize(df['attributes']))
df = df.join(pd.json_normalize(df['hours']))

# Drop the attributes and hours columns containing JSON data
df = df.drop(columns=['attributes', 'hours'])

### II. Data Exploration

### III. Feature Engineering


In [6]:
# Function to parse an hours string and return number of hours open
# e.g. 10:00-21:00 -> 11 hours

def parse_hours(day_hours_str):
    if pd.isna(day_hours_str):
        return 0
    
    time_endpoints = str(day_hours_str).split('-')

    if time_endpoints[0] == time_endpoints[1]:
        # 0:0-0:0
        return 24
    
    start_time = time.strptime(time_endpoints[0], "%H:%M")
    end_time = time.strptime(time_endpoints[1], "%H:%M")

    # account for edge cases in data where we have 10-1, which is technically 10am-1am
    et_hour = (24 + end_time.tm_hour) if end_time.tm_hour < start_time.tm_hour else end_time.tm_hour
    
    start_time_td = timedelta(hours=start_time.tm_hour, minutes=start_time.tm_min)
    end_time_td = timedelta(hours=et_hour, minutes=end_time.tm_min)

    duration = end_time_td - start_time_td

    return duration.total_seconds() / 3600

In [7]:
# Create new feature (total_open_hours) by combining all individual day hours

total_hours_arr = []
count_neg = 0
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

for ind in df.index:
    total_hours = 0

    for day in days:
        day_hours_str = df[day][ind]
        day_hours = parse_hours(day_hours_str)
        total_hours += day_hours
    
    total_hours_arr.append(total_hours)

df['total_open_hours'] = total_hours_arr

# Drop all the individual day hours columns
df = df.drop(columns=days)

# TODO show example of total_open_hours before dropping days?

In [8]:
# Remove all features except a handful

df = df.filter(['total_open_hours', 'RestaurantsTakeOut', 'RestaurantsDelivery', 'Alcohol', 'latitude', 'longitude', 'stars'])

In [9]:
# Impute missing values with false

df['RestaurantsTakeOut'] = df['RestaurantsTakeOut'].fillna('False')
df['RestaurantsDelivery'] = df['RestaurantsDelivery'].fillna('False')
df['Alcohol'] = df['Alcohol'].fillna('False')

In [10]:
# Convert alcohol column to true/false values only

def alcohol_tf(val):
    if 'beer_and_wine' in val or 'full_bar' in val:
        return True
    else :
        return False

df['Alcohol_TF'] = df['Alcohol'].apply(alcohol_tf)

In [11]:
# Replace all 'None' values with False
df.replace('None', 'False', inplace=True)

# Convert all string representations of T/F to boolean values
df.replace({'True': True, 'False': False}, inplace=True)

# Drop alcohol feature (already feature engineered it)
df.drop(columns=['Alcohol'], inplace = True)

# Rename alcohol T/F feature column
df = df.rename(columns={'Alcohol_TF':'Alcohol'})

# Reset the index to 0, 1, ... - it changed after all the drops and modifications
df = df.reset_index()

In [12]:
# Apply KNN to find k nearest restaurants (by latitude/longitude)

# Create new feature (avg_star_rating) averaging the star rating of the k nearest restaurants

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

location_df = df[['latitude', 'longitude', 'stars']]

stars = location_df['stars']
location_df = location_df.drop(columns='stars')

scaler = StandardScaler()
location_df = pd.DataFrame(scaler.fit_transform(location_df), columns=location_df.columns)

neigh = NearestNeighbors(n_neighbors=51, n_jobs=-1)

neigh.fit(location_df[['latitude', 'longitude']])

distances, indices = neigh.kneighbors(location_df[['latitude', 'longitude']])

for i in range(len(location_df)):
    location_df.loc[i, 'avg_star_rating'] = stars.iloc[indices[i]].mean()

In [13]:
# Add stacking feature to main df
df['stack_1'] = location_df['avg_star_rating']

# Drop latitude/longitude feature columns (no need anymore)
df.drop(columns=['latitude', 'longitude'], inplace=True)

In [14]:
# Train Decision Tree Regressor for second stack as a feature in dataframe
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

clf = DecisionTreeRegressor(random_state=0)
X = df.loc[:, df.columns != 'stars']
y = df.loc[:, 'stars']

# Fitting the model based on max_depth = 4 because we have 4 features, ideally splitting on each one
regressor = DecisionTreeRegressor(random_state=0, max_depth=4)
regressor.fit(X, y) 

y_pred = regressor.predict(X) 
df['stack_2'] = y_pred

# Reorder df columns
df = df[['RestaurantsTakeOut', 'RestaurantsDelivery', 'Alcohol', 'total_open_hours', 'stack_1', 'stack_2', 'stars']]

In [15]:
# Final df for model looks like this (features and labels)
df.head()

,RestaurantsTakeOut,RestaurantsDelivery,Alcohol,total_open_hours,stack_1,stack_2,stars
0,False,False,False,23.0,3.960784,3.927685,4.0
1,True,True,True,53.0,3.225490,3.176283,2.0
2,False,False,False,100.0,3.235294,3.176283,1.5
3,True,True,False,60.0,3.931373,3.927685,4.0
4,True,False,True,58.0,3.284314,3.265017,2.5


In [16]:
# Train models with all combinations of stacks/no stacks, so define variables for the same
labels = df['stars']
features_without_stacks = df.drop(columns=['stack_1', 'stack_2', 'stars'])
features_stack1 = df.drop(columns=['stack_2', 'stars'])
features_stack2 = df.drop(columns=['stack_1', 'stars'])
features_all_stacks = df.drop(columns=['stars'])

In [17]:
# Linear Regression

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score

# features_without_stacks

X = features_without_stacks
y = labels
reg = LinearRegression()

# 10-fold cross validation on LinearRegression() Model
scores = cross_val_score(reg, X, y, cv=10)
print('R^2 of Linear Regression with features_without_stacks:', np.mean(scores))

# features_stack1

X = features_stack1
y = labels

reg = LinearRegression()

scores = cross_val_score(reg, X, y, cv=10)
print('R^2 of Linear Regression with features_stack1:', np.mean(scores))

# features_stack2

X = features_stack2
y = labels
reg = LinearRegression()

# 10-fold cross validation on LinearRegression() Model
scores = cross_val_score(reg, X, y, cv=10)
print('R^2 of Linear Regression with nfeatures_stack2:', np.mean(scores))

# features_all_stacks

X = features_all_stacks
y = labels
reg = LinearRegression()

# 10-fold cross validation on LinearRegression() Model
scores = cross_val_score(reg, X, y, cv=10)
print('R^2 of Linear Regression with features_all_stacks:', np.mean(scores))

R^2 of Linear Regression with features_without_stacks: -0.0007713364626037889
R^2 of Linear Regression with features_stack1: 0.09931034573759065
R^2 of Linear Regression with nfeatures_stack2: 0.10072682147360687
R^2 of Linear Regression with features_all_stacks: 0.10077162630391703


In [18]:
# Decision Tree Regressor

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

# features_without_stacks

X = features_without_stacks
y = labels
reg_dt = DecisionTreeRegressor(random_state=0, max_depth=4)

# 10-fold cross validation on LinearRegression() Model
scores = cross_val_score(reg_dt, X, y, cv=10)
print('R^2 of Decision Tree Regressor with features_without_stacks:', np.mean(scores))

# features_stack1

X = features_stack1
y = labels
reg_dt = DecisionTreeRegressor(random_state=0, max_depth=4)

# 10-fold cross validation on LinearRegression() Model
scores = cross_val_score(reg_dt, X, y, cv=10)
print('R^2 Decision Tree Regressor with features_stack1:', np.mean(scores))

# features_stack2

X = features_stack2
y = labels
reg_dt = DecisionTreeRegressor(random_state=0, max_depth=4)

# 10-fold cross validation on LinearRegression() Model
scores = cross_val_score(reg_dt, X, y, cv=10)
print('R^2 of Decision Tree Regressor with features_stack2:', np.mean(scores))

# features_all_stacks

X = features_all_stacks
y = labels
reg_dt = DecisionTreeRegressor(random_state=0, max_depth=4)

# 10-fold cross validation on LinearRegression() Model
scores = cross_val_score(reg_dt, X, y, cv=10)
print('R^2 of Decision Tree Regressor with features_all_stacks:', np.mean(scores))

R^2 of Decision Tree Regressor with features_without_stacks: -0.002657190982240931
R^2 Decision Tree Regressor with features_stack1: 0.09791090772762898
R^2 of Decision Tree Regressor with features_stack2: 0.09953694893616721
R^2 of Decision Tree Regressor with features_all_stacks: 0.09795853583354919


In [19]:
# Neural Nets Regressor

# setting up neural network, with pipeline that scales the data
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import r2_score

mlp_reg = MLPRegressor()
scaler = StandardScaler()
pipeline = Pipeline([('scaler', scaler), ('mlp', mlp_reg)])
param_grid = {
    'mlp__activation': ['logistic', 'tanh', 'relu']
}
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs = -1)

# neural net training with features_without_stacks, 5-fold cross validation
predictions = cross_val_predict(grid_search, features_without_stacks, labels, cv=5)
print("R^2 of Neural Nets MLP Regressor features_without_stacks:", r2_score(predictions, labels))

# neural net training with features_stack1, 5-fold cross validation
predictions = cross_val_predict(grid_search, features_stack1, labels, cv=5)
print("R^2 of Neural Nets MLP Regressor with features_stack1:", r2_score(predictions, labels))

# neural net training with features_stack2, 5-fold cross validation
predictions = cross_val_predict(grid_search, features_stack2, labels, cv=5)
print("R^2 of Neural Nets MLP Regressor with features_stack2:", r2_score(predictions, labels))

# neural net training with features_all_stacks, 5-fold cross validation
predictions = cross_val_predict(grid_search, features_all_stacks, labels, cv=5)
print("R^2 of Neural Nets MLP Regressor with features_all_stacks:", r2_score(predictions, labels))

R^2 of Neural Nets MLP Regressor features_without_stacks: -28.021152152437928


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


KeyboardInterrupt: 

In [ ]:
# KNN Classifier

from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

num_bins = 9
bin_labels = [f'Category_{i}' for i in range(num_bins)]
labels_categorical = pd.cut(labels, bins=num_bins, labels=bin_labels)

# Features Without Stacks, 5-fold cross validation
knn = KNeighborsClassifier(n_neighbors=350)
cv_scores = cross_val_score(knn, features_without_stacks, labels_categorical, cv=5)
# Calculate the accuracy
avg_score = np.mean(cv_scores) * 100.0
print('Average accuracy of KNN Classifier with features_without_stacks: ' + str(avg_score) + "%")

# With only Stack 1, 5-fold cross validation
cv_scores = cross_val_score(knn, features_stack1, labels_categorical, cv=5)
avg_score = np.mean(cv_scores) * 100.0
print('Average accuracy of KNN Classifier with features_stack1: ' + str(avg_score) + "%")

# With only Stack 2, 5-fold cross validation
cv_scores = cross_val_score(knn, features_stack2, labels_categorical, cv=5)
avg_score = np.mean(cv_scores) * 100.0
print('Average accuracy of KNN Classifier with features_stack2: ' + str(avg_score) + "%")

# With both Stack 1 and Stack 2, 5-fold cross validation
cv_scores = cross_val_score(knn, features_all_stacks, labels_categorical, cv=5)
avg_score = np.mean(cv_scores) * 100.0
print('Average accuracy of KNN Classifier with features_all_stacks: ' + str(avg_score) + "%")

In [ ]:
# Decision Tree Classifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

# Convert labels from floats to categorical classes (for classification)
# 1.0, 1.5, ..., 4.5, 5.0 --> 9 bins
num_bins = 9
bin_labels = [f'Category_{i}' for i in range(num_bins)]
labels_categorical = pd.cut(labels, bins=num_bins, labels=bin_labels)

clf_criterion = 'gini'

clf = DecisionTreeClassifier(criterion=clf_criterion, random_state=0)

# 5-fold cross validation
scores = cross_val_score(clf, features_without_stacks, labels_categorical, cv=5)
print("Accuracy of Decision Tree Classifier with features_without_stacks:", scores.mean())

# 5-fold cross validation
scores = cross_val_score(clf, features_stack1, labels_categorical, cv=5)
print("Accuracy of Decision Tree Classifier with features_stack1:", scores.mean())

# 5-fold cross validation
scores = cross_val_score(clf, features_stack2, labels_categorical, cv=5)
print("Accuracy of Decision Tree Classifier with features_stack2:", scores.mean())

# 5-fold cross validation
scores = cross_val_score(clf, features_all_stacks, labels_categorical, cv=5)
print("Accuracy of Decision Tree Classifier with features_all_stacks:", scores.mean())

In [ ]:
# Neural Nets Classifier

from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline

# Pipeline the data
scaler = StandardScaler()
nn = MLPClassifier(hidden_layer_sizes=30, activation='logistic')
pipe = Pipeline(steps=[('scaler', scaler), 
                      ('mlp', nn)])

grid_search = GridSearchCV(pipe, param_grid, cv=5)
grid_search.fit(features_without_stacks, labels_categorical)

# With no stacks, 5-fold cross validation
cv_scores = cross_val_score(grid_search, features_without_stacks, labels_categorical, cv=5)
print("Accuracy of Neural Nets MLP Classifier with features_without_stacks:", str(cv_scores.mean() * 100) + "%")

# With Stack 1 only, 5-fold cross validation
grid_search.fit(features_stack1, labels_categorical)
cv_scores = cross_val_score(grid_search, features_stack1, labels_categorical, cv=5)
print("Accuracy of Neural Nets MLP Classifier with features_stack1:", str(cv_scores.mean() * 100) + "%")

# With Stack 2 only, 5-fold cross validation
grid_search.fit(features_stack2, labels_categorical)
cv_scores = cross_val_score(grid_search, features_stack2, labels_categorical, cv=5)
print("Accuracy of Neural Nets MLP Classifier with features_stack2:", str(cv_scores.mean() * 100) + "%")

# With Stacks 1 and 2, 5-fold cross validation
grid_search.fit(features_all_stacks, labels_categorical)
cv_scores = cross_val_score(grid_search, features_all_stacks, labels_categorical, cv=5)
print("Accuracy of Neural Nets MLP Classifier with features_all_stacks:", str(cv_scores.mean() * 100) + "%")

In [ ]:
# K-Means Classifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import random
from sklearn.manifold import MDS
import matplotlib.pyplot as plt

def scatter(data, labels, numPoints = 300):

    numEntries = data.shape[0]
    start = random.randint(0, numEntries - numPoints)
    end = start + numPoints
    data = data.iloc[start:end, :]
    labels = labels.iloc[start:end]
    
    mds = MDS(n_components=2)
    mds_data = mds.fit_transform(data)
    plt.scatter(mds_data[:, 0], mds_data[:, 1], c=labels, s=50)
    plt.show()

test_curr_start = 0
test_curr_end = int(len(features_stack2) / 10)
increment = int(len(features_stack2) / 10)
mse = 0
total_sh_score = 0

for i in range(10):
    print(f"Fold {i+1}:")
    
    # partition data into train_set and test_set
    a = features_stack2.iloc[:test_curr_start, :].values
    b = features_stack2.iloc[test_curr_end:, :].values
    X_train = np.concatenate((a, b))
    X_test = features_stack2.iloc[test_curr_start:test_curr_end, :].values
    a = labels.iloc[:test_curr_start]
    b = labels.iloc[test_curr_end:]
    y_train = np.concatenate((a, b))
    y_test = labels.iloc[test_curr_start:test_curr_end].values

    kmeans = KMeans(n_clusters=9)
    curr_clustering = kmeans.fit_predict(X_train)

    sh_score = silhouette_score(X_train, curr_clustering)
    total_sh_score += sh_score
    print(f"\tsilhouette score: {sh_score}")
    # scatter(pd.DataFrame(X_train), pd.Series(curr_clustering)) # uncomment to see clustering for 300 random data points

    trained_df = pd.DataFrame(X_train)
    trained_df['k_means_cluster'] = curr_clustering
    trained_df['stars'] = y_train
    
    # calculate average star prediction for each cluster
    cluster_preds = trained_df.groupby('k_means_cluster', as_index=False)['stars'].mean()['stars']
    pred_clustering = kmeans.predict(X_test)

    predictions = []
    for i in pred_clustering:
        predictions.append(cluster_preds[i])

    curr_mse = mean_squared_error(y_test, predictions)
    print("\tcurr_mse:", curr_mse)
    mse += curr_mse
    test_curr_start += increment
    test_curr_end += increment
    
print("\nAverage sh score", total_sh_score/10)
print("Average mse", mse/10)

In [ ]:
# Final predictions for each cluster, trained on features_stack2
kmeans = KMeans(n_clusters=9)
curr_clustering = kmeans.fit_predict(features_stack2)

# calculate average star prediction for each cluster
cluster_preds = trained_df.groupby('k_means_cluster', as_index=False)['stars'].mean()['stars']


# The star prediction for a new datapoint will be made based on the cluster that the new datapoint is in. 
# cluster_preds stores star predictions for each cluser